# CSCI 4253 Final Project
# Machine Learning Prototype

## Import Data
Please change the configuration below for your own local settings. Currently this is importing a CSV file from an AWS S3 bucket.

In [ ]:
import io
import boto3
import pandas as pd

s3 = boto3.client('s3')
file = s3.get_object(Bucket = '', Key = '.csv')
df_all = pd.read_csv(io.BytesIO(file['Body'].read()))
print("Done!")

## Process the data
Remove or configure any additional input features below

In [ ]:
# E.g. .sample(10000) to get 10,000 random data points

## Split the data
Split the data into training and testing. The model will train on the "training" data and then to test the accuracy we check how well the model performed in predicting the testing data.

For now, 80% of the data will be used for training.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size =0.2, random_state=39)

## Train the model
For our model, since we are predicting a "hotness" score for an input. That is, given a plethora of data about a song, our model will take that and give a score in [0,1] which will represent how popular a song is.

Note we are comparing two different models below.

Below, we are using multivariate linear regression for a couple of different reasons. Mainly because since it is regression, we can use continous values for our output. Also unlike normal linear regression, we can create non-linear relationships.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [1]:
from sklearn.linear_model import LinearRegression
# Do training here

Below, we are using logistic regression, where we classify a song as popular if its hotness >= x where x is between 0 and 1 (e.g. x = 0.6). This would tell us not only if a song will be popular, but give us a confidence score, similar to hotness. 

Please see: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html for more information about LogisticRegression.

In [2]:
from sklearn.linear_model import LogisticRegression
# Do training here

## Test the models

In [ ]:
from sklearn.metrics import accuracy_score


y_predict = .predict(X_test)
accuracy_score(y_test, y_predict)

## Metrics